##  A tour of using the revised DNAStack WES Server

This will work with Thousand Genomes bam files. Even though the Thousand Genomes bam files are publically available care was taken in the following not to show the whole contents of the run details returned by the WES server. The response contains signed URLs. In other cases that would allow access to controlled access data - albeit for a limited time.

This function shows some details of the run, but not the possibly sensitive stuff.

In [14]:
def getMinimalRunLog(wesClient, run_id):
    rundetails = wesClient.GetRunLog(run_id)
    print(rundetails['run_id'])
    print(rundetails['state'])
    print(rundetails['outputs'])

We can now use that function to minimal details of a run submitted via FASPScript4.py. That script submits a workflow to run an MD5 checksum on a Thousand Genomes bam file. It's output should just be that checksum.

In [16]:
from fasp.workflow import DNAStackWESClient
wesClient = DNAStackWESClient('~/.keys/dnastack_wes_credentials.json')
getMinimalRunLog('523abb68-44bd-4040-b7b0-bfc2164f8917')

523abb68-44bd-4040-b7b0-bfc2164f8917
COMPLETE
{'md5Sum.md5': ''}


That indicates an empty response. Though the task appeared to complete successfully we have no result. How can we check what happened? Links to stdout and stderr are available in the response.

In [2]:
log = rundetails['task_logs'][0]
for stdx in ['stdout','stderr']:
    print('{} is at {}\n'.format(stdx, log[stdx]))


stdout is at https://workspaces-wes.prod.dnastack.com/ga4gh/wes/v1/runs/523abb68-44bd-4040-b7b0-bfc2164f8917/logs/task/md5Sum.calculateMd5Sum/0/stdout

stderr is at https://workspaces-wes.prod.dnastack.com/ga4gh/wes/v1/runs/523abb68-44bd-4040-b7b0-bfc2164f8917/logs/task/md5Sum.calculateMd5Sum/0/stderr



Following those links downloads empty files. That is the same as the previous version of the server. It was suggested that DRS would be a suitable way to make the files available.

The absence of the files means we don't have access to information that would be useful for troubleshooting. The guess is that it is the same problem as previously.

Trying another workflow. 

The function runGWASWorkflowTest encapsulates the GWAS Workflow from the 2020 GA4GH Plenary. It runs it on files accessed via Google Cloud URIs rather than DRS. This lets us check a known workflow.

In [3]:
wesClient.runGWASWorkflowTest()

'0fcdbc2e-9990-4d7c-916a-41d363985aeb'

In [17]:
getMinimalRunLog(wesClient, '0fcdbc2e-9990-4d7c-916a-41d363985aeb')

0fcdbc2e-9990-4d7c-916a-41d363985aeb
COMPLETE
{'gwas.logistic': 'gs://workspaces-cromwell-execution/gwas/0fcdbc2e-9990-4d7c-916a-41d363985aeb/call-run_gwas/CCDG_13607_B01_GRM_WGS_2019-02-19_chr21.recalibrated_variants.assoc.logistic', 'gwas.manhattan_plot': 'gs://workspaces-cromwell-execution/gwas/0fcdbc2e-9990-4d7c-916a-41d363985aeb/call-create_plot/CCDG_13607_B01_GRM_WGS_2019-02-19_chr21.recalibrated_variants.png'}


In this case the run has produced some output files which look promising. How we access those files is still an open question. See discussion elsewhere about whether a companion DRS server, or even a DRS server accessed via the WES server and accessed under the same credentials would be a useful addition to WES.